In [3]:
# import asyncio
from lowe.locations.lookup import search, generate_df_json
from lowe.acs.ACSClient import ACSClient
import pandas as pd

import plotly.express as px
import plotly.figure_factory as ff

**Temp Scratch**

In [2]:
client = ACSClient()
await client.initialize()

In [9]:
for city in ['cathedral city, ca', 'coachella, ca', 'desert hot springs, ca', 'indian wells, ca', 'indio, ca', 'la quinta, ca', 'palm desert, ca', 'palm springs, ca', 'rancho mirage, ca']:
    for table in ["S0801", "S2801", "S1901", "S1501", "DP05"]:
        fip_code2 = search(city, codetype="city", search_on="name").iloc[0][1]

        locs = [{'city': fip_code2}]
        try:
            resp = await client.get_acs(vars = [table],
                                start_year='2019',
                                end_year='2019',
                                location=locs,
                                estimate="1")
            print(city, table)
        except:
            print("Failed")
        

Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
indio, ca S0801
indio, ca S2801
indio, ca S1901
indio, ca S1501
indio, ca DP05
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed


In [11]:
for city in ['cathedral city, ca', 'coachella, ca', 'desert hot springs, ca', 'indian wells, ca', 'indio, ca', 'la quinta, ca', 'palm desert, ca', 'palm springs, ca', 'rancho mirage, ca']:
    for table in ["CG00ORG11", "S0504", "S0505", "B06004B"]: #"S0801", "S2801", "S1901", "S1501", "DP05"
        fip_code2 = search(city, codetype="city", search_on="name").iloc[0][1]

        locs = [{'city': fip_code2}]
        try:
            resp = await client.get_acs(vars = [table],
                                start_year='2019',
                                end_year='2019',
                                location=locs,
                                estimate="1")
            print(city, table)
        except:
            print("Failed")
        

Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
indio, ca S0504
indio, ca S0505
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed


In [10]:
for city in ['san bernardino, ca', 'riverside, ca']:
    for table in ["S0801", "S2801", "S1901", "S1501", "DP05"]:
        fip_code2 = search(city, codetype="city", search_on="name").iloc[0][1]

        locs = [{'city': fip_code2}]
        try:
            resp = await client.get_acs(vars = [table],
                                start_year='2019',
                                end_year='2019',
                                location=locs,
                                estimate="1")
            print(city, table)
        except:
            print("Failed")
        

san bernardino, ca S0801
san bernardino, ca S2801
san bernardino, ca S1901
san bernardino, ca S1501
san bernardino, ca DP05
Failed
Failed
Failed
Failed
Failed


In [18]:
fip_code2 = search('indio, ca', codetype="city", search_on="name").iloc[0][1]

locs = [{'city': fip_code2}]
resp = await client.get_acs(vars = ['S1501'],
                        start_year='2019',
                        end_year='2019',
                        location=locs,
                        estimate="1")

In [28]:
resp.iloc[:, 74]

year
2019    1879
Name: EDUCATIONAL ATTAINMENT Estimate Male AGE BY EDUCATIONAL ATTAINMENT Population 25 years and over Associate's degree, dtype: object

--- 
helper functions, run first

In [ ]:
def city_search(cities):
   # there is not search that returns the df?

    all_loc = []

    for city in cities:
        df = generate_df_json(codetype="city")
        df = df.astype(str)  # Convert all cols to string
        df = df[df["name"].str.contains(city)]
        if df.iloc[0][0] != df.iloc[-1][0]:
            raise ValueError("Returned more than 1 city in city search.")
        fips = df.iloc[0]['fips']
        loc = {
            "state": fips[0:2],
            "city": fips[2:]
        }
        all_loc.append(loc)
    return all_loc

In [ ]:
async def acs_data(loc_fips: object, start_year: str, table='S1901', end_year: str = None):
    """
    takes result of city_search loc_fips and gets the data
    """
    client = ACSClient()
    await client.initialize()
    
    if end_year == None:
        end_year = start_year
    
    raw_resp = await client.get_acs(vars = [table],
                     start_year=start_year,
                     end_year=end_year,
                     location=loc_fips,
                     estimate="5")
    
    return raw_resp

---

In [ ]:
pri_color = "#961a30"
sec_color = '#e7c8ae'

---

In [ ]:
# Fig 8

async def total_household_income(cities: list = ['cathedral city, ca', 'coachella, ca', 'desert hot springs, ca', 'indian wells, ca', 'indio, ca', 'la quinta, ca', 'palm desert, ca', 'palm springs, ca', 'rancho mirage, ca'], save: bool = False, save_path: str = None):
    """
    Parameters
    ----------
    city: name of the city eg. 
    target
    save: bool
    True or False, whether or not you want to save
    save_path: str
    Path to save the file to
    """
#     target_col = "INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Households Total"
#     new_col_name = "Total Income"
    
#     INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Families Total * INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Families Mean income (dollars)
    
    cols = {
        "INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Families Total": "Families Total",
        "INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Families Mean income (dollars)": "Income Per"
    }
    
    loc_fips = city_search(cities)
    
    resp = await acs_data(loc_fips, "2019")
    
    col_sub = [*list(cols.keys()), "state", "city"]
    resp = resp[col_sub]
    resp = resp.rename(columns = cols)
    
    resp["Total Income"] = resp["Families Total"].astype(str).astype(int) * resp["Income Per"].astype(str).astype(int)
    
    
    # Following is unique to each section
    
    categ = resp["city"]
    value = resp["Total Income"]

    
    plot_df = pd.DataFrame({'city': categ, 'Value': value})
    
    fig = px.bar(plot_df, x="city", y="Value")
    fig.update_xaxes(type='category')
    
    fig.update_layout(
        font_family="Glacial Indifference",
        font_color="black",
        title_font_family="Glacial Indifference",
        title_font_color="black",
        legend_title_font_color="black", 
         template="simple_white"
    )
    
    fig.update_traces(marker_color=pri_color)
    
    if save:
        fig.write_image(save_path, format='png')
    return fig

In [ ]:
# Fig 9 
async def median_household_income(cities: list = ['cathedral city, ca', 'coachella, ca', 'desert hot springs, ca', 'indian wells, ca', 'indio, ca', 'la quinta, ca', 'palm desert, ca', 'palm springs, ca', 'rancho mirage, ca'], save: bool = False, save_path: str = None):
    """
    DO NOT PASS CITIES PARAM (is a list of them); if one city just pass as list of 1
    Parameters
    ----------
    city: name of the city eg. 
    target
    save: bool
    True or False, whether or not you want to save
    save_path: str
    Path to save the file to
    """
    target_col = "INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Families Median income (dollars)"
    new_col_name = "Median Income"
    
    
    
    loc_fips = city_search(cities)
    
    resp = await acs_data(loc_fips, "2019")
    
    col_sub = [target_col, "state", "city"]
    resp = resp[col_sub]
    resp = resp.rename(columns = {
        target_col: new_col_name
    })

    
    # Following is unique to each section
    
    categ = resp["city"]
    value = resp["Median Income"].astype(str).astype(int)
    
    plot_df = pd.DataFrame({'Type': categ, 'Value': value})
    
    fig = px.bar(plot_df, x="Type", y="Value")
    fig.update_xaxes(type='category')
    
    fig.update_layout(
        font_family="Glacial Indifference",
        font_color="black",
        title_font_family="Glacial Indifference",
        title_font_color="black",
        legend_title_font_color="black", 
         template="simple_white"
    )
    
    fig.update_traces(marker_color=pri_color)
    
    if save:
        fig.write_image(save_path, format='png')
    return fig

In [ ]:
# Fig 10
async def household_income_byClass(cities: list = ['coachella, ca', 'desert hot springs, ca'], save: bool = False, save_path: str = None):
    """
    DO NOT PASS CITIES PARAM (is a list of them); if one city just pass as list of 1
    Parameters
    ----------
    city: name of the city eg. 
    target
    save: bool
    True or False, whether or not you want to save
    save_path: str
    Path to save the file to
    """
#     target_cols = "INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Households Total"
#     new_col_name = "Total Income"
    
    cols = {
        # old col name: new name
        "INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Nonfamily households Total Less than $10,000" : "Less than $10,000",
        "INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Nonfamily households Total $10,000 to $14,999": "$10,000 to $14,999",
        "INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Nonfamily households Total $15,000 to $24,999": "$15,000 to $24,999",
        "INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Nonfamily households Total $25,000 to $34,999": "$25,000 to $34,999",
        "INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Nonfamily households Total $35,000 to $49,999": "$35,000 to $49,999",
        "INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Nonfamily households Total $50,000 to $74,999": "$50,000 to $74,999",
        "INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Nonfamily households Total $75,000 to $99,999": "$75,000 to $99,999",
        "INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Nonfamily households Total $100,000 to $149,999": "$100,000 to $149,999",
        "INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Nonfamily households Total $150,000 to $199,999": "Total $150,000 to $199,999",
        "INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) Estimate Nonfamily households Total $200,000 or more": "$200,000 or more"
    }
    
    
    
    loc_fips = city_search(cities)
    
    resp = await acs_data(loc_fips, "2019")
    
    col_sub = [*list(cols.keys()), "state", "city"]
    resp = resp[col_sub]
    resp = resp.rename(columns = cols)
    
    
    # Following is unique to each section
    
    categ = resp.columns[0:10]
    bar_value = resp.iloc[1][0:10].astype(str).astype(float)
    line_value = resp.iloc[0][0:10].astype(str).astype(float)
    
    bar_df = pd.DataFrame({'Type': categ, 'Percentage': bar_value})
    line_df = pd.DataFrame({'Type': categ, 'Percentage': line_value})

    fig = px.line(line_df, x="Type", y="Percentage", color=px.Constant("Coachella Valley"), color_discrete_sequence=[sec_color])
    fig.add_bar(x=bar_df["Type"], y=bar_df["Percentage"], name="Desert Hot Spring")
    
        
    fig.update_layout(
        font_family="Glacial Indifference",
        font_color="black",
        title_font_family="Glacial Indifference",
        title_font_color="black",
        legend_title_font_color="black", 
         template="simple_white"
    )
    
    fig.update_traces(marker_color=pri_color)
    
    
    if save:
        fig.write_image(save_path, format='png')
    return fig

In [ ]:
# Fig 7
async def household_with_internet(cities: list = ['cathedral city, ca', 'coachella, ca', 'desert hot springs, ca', 'indian wells, ca', 'indio, ca', 'la quinta, ca', 'palm desert, ca', 'palm springs, ca', 'rancho mirage, ca'], save: bool = False, save_path: str = None):
    """
    DO NOT PASS CITIES PARAM (is a list of them); if one city just pass as list of 1
    Parameters
    ----------
    city: name of the city eg. 
    target
    save: bool
    True or False, whether or not you want to save
    save_path: str
    Path to save the file to
    """
    target_cols = "TYPES OF COMPUTERS AND INTERNET SUBSCRIPTIONS Estimate Percent Total households TYPE OF INTERNET SUBSCRIPTIONS With an Internet subscription: Broadband of any type"
    new_col_name = "Percent of Total Households with Broadband"
    
    
    loc_fips = city_search(cities)
    
    resp = await acs_data(loc_fips, "2019", 'S2801')
    
    col_sub = [target_cols, "state", "city"]
    resp = resp[col_sub]
    resp = resp.rename(columns = {target_cols: new_col_name})
    
    
    
    # Following is unique to each section
    
    categ = resp["city"]
    value = resp["Percent of Total Households with Broadband"].astype(str).astype(float)
    
    plot_df = pd.DataFrame({'Type': categ, 'Value': value})
    
    fig = px.bar(plot_df, x="Type", y="Value")
    fig.update_xaxes(type='category')
    
    
    fig.update_layout(
        font_family="Glacial Indifference",
        font_color="black",
        title_font_family="Glacial Indifference",
        title_font_color="black",
        legend_title_font_color="black", 
         template="simple_white"
    )
    
    fig.update_traces(marker_color=pri_color)
    
    
    if save:
        fig.write_image(save_path, format='png')
    return fig

In [ ]:
# Fig 6
async def race_group_distribution(cities: list = ['desert hot springs, ca'], save: bool = False, save_path: str = None):
    """
    DO NOT PASS CITIES PARAM (is a list of them); if one city just pass as list of 1
    Parameters
    ----------
    city: name of the city eg. 
    target
    save: bool
    True or False, whether or not you want to save
    save_path: str
    Path to save the file to
    """
    target_cols = "TYPES OF COMPUTERS AND INTERNET SUBSCRIPTIONS Estimate Percent Total households TYPE OF INTERNET SUBSCRIPTIONS With an Internet subscription: Broadband of any type"
    new_col_name = "Percent of Total Households with Broadband"
    
    
    cols = {
        # old col name: new name
        "ACS DEMOGRAPHIC AND HOUSING ESTIMATES Percent RACE Total population One race White": "White alone",
        "ACS DEMOGRAPHIC AND HOUSING ESTIMATES Percent HISPANIC OR LATINO AND RACE Total population Not Hispanic or Latino White alone": "White Alone, not Hispanic or Latino",
        "ACS DEMOGRAPHIC AND HOUSING ESTIMATES Percent HISPANIC OR LATINO AND RACE Total population Hispanic or Latino (of any race)": "Hispanic or Latino",
        "ACS DEMOGRAPHIC AND HOUSING ESTIMATES Percent RACE Total population One race Black or African American": "Black or African American alone",
        "ACS DEMOGRAPHIC AND HOUSING ESTIMATES Percent HISPANIC OR LATINO AND RACE Total population Not Hispanic or Latino American Indian and Alaska Native alone": "American Indian and Alaska Native alone",
        "ACS DEMOGRAPHIC AND HOUSING ESTIMATES Percent RACE Total population One race Asian": "Asian alone",
        "ACS DEMOGRAPHIC AND HOUSING ESTIMATES Percent RACE Total population One race Native Hawaiian and Other Pacific Islander": "Native Hawaiian and Other Pacific Islander alone",
        "ACS DEMOGRAPHIC AND HOUSING ESTIMATES Percent RACE Total population Two or more races": "Two or more races",
    }
    
    
    loc_fips = city_search(cities)
    
    resp = await acs_data(loc_fips, "2019", "DP05")
    
    col_sub = [*list(cols.keys()), "state", "city"]
    resp = resp[col_sub]
    resp = resp.rename(columns = cols)
    
    
    
    # Following is unique to each section
    
    categ = resp.columns[0:8]
    value = resp.iloc[0][0:8].astype(str).astype(float)
    

    plot_df = pd.DataFrame({'Race': categ, 'Percentage': value})
    
    fig = px.bar(plot_df, x="Race", y="Percentage", text='Percentage')
    fig.update_xaxes(type='category')
    
        
    fig.update_layout(
        font_family="Glacial Indifference",
        font_color="black",
        title_font_family="Glacial Indifference",
        title_font_color="black",
        legend_title_font_color="black", 
         template="simple_white"
    )
    
    fig.update_traces(marker_color=pri_color)
    
    
    if save:
        fig.write_image(save_path, format='png')
    return fig
    

# Plot Outputs

In [ ]:
ans = await total_household_income()
ans

In [ ]:
ans = await median_household_income()
ans

In [ ]:
ans = await race_group_distribution()
ans

In [ ]:
ans = await household_with_internet()
ans

In [ ]:
ans = await household_income_byClass()
ans

---

In [ ]:
# table 1
async def residence_and_work_loc(cities: list = ['desert hot springs, ca'], save: bool = False, save_path: str = None):
    """
    Parameters
    ----------
    city: name of the city eg. 
    target
    save: bool
    True or False, whether or not you want to save
    save_path: str
    Path to save the file to
    """
    cols = {
        # old col name: new name
        "COMMUTING CHARACTERISTICS BY SEX Estimate Total PERCENT ALLOCATED Place of work": "% Allocated Place of Work",
        "COMMUTING CHARACTERISTICS BY SEX Estimate Total Workers 16 years and over PLACE OF WORK Not living in 12 selected states": "Count, live outside state",
        "COMMUTING CHARACTERISTICS BY SEX Estimate Total Workers 16 years and over PLACE OF WORK Worked in state of residence Worked outside county of residence": "Count, outside of county",
        "COMMUTING CHARACTERISTICS BY SEX Estimate Total Workers 16 years and over PLACE OF WORK Worked in state of residence Worked in county of residence": "Count, inside county",
    }
    
    
    loc_fips = city_search(cities)
    
    resp = await acs_data(loc_fips, "2019", "S0801")
    
    col_sub = [*list(cols.keys()), "state", "city"]
    resp = resp[col_sub]
    resp = resp.rename(columns = cols)
    
    ans = [["Living/Working", resp.iloc[0]['city']], [resp.columns[1], resp.iloc[0][1]], [resp.columns[2], resp.iloc[0][2]], [resp.columns[3], resp.iloc[0][3]]]
    fig = ff.create_table(ans, colorscale=[[0, pri_color], [.5, '#ffffff'], [1, '#ffffff']])
    
    return fig
    

In [ ]:
table1 = await residence_and_work_loc()
table1.show()